In [1]:
import os
import json
from pathlib import Path

from dotenv import load_dotenv
from llama_cpp import Llama
from transformers import AutoTokenizer, PreTrainedTokenizer

from pymongo import MongoClient
from datetime import datetime, timedelta

from django.shortcuts import render
from django.http import JsonResponse, FileResponse

from langchain_openai import ChatOpenAI
from pymongo import MongoClient

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
import re

# retriever 사용을 위한 라이브러리
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

c:\Users\RMARKET\anaconda3\envs\var-ser\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
input_string = '머리카락 길이에 대해서 알려줘'

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY_sesac')
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    api_key=api_key
)

prompt = PromptTemplate.from_template('''
{message}

금융과 무관한 내용을 금융과 억지로 연결 짓지 마세요. 
금융(예: 주식, 투자, 은행, 대출, 경제 등)과 직접 관련이 있는 경우에만 "True"를 반환하고, 그렇지 않다면 "False"를 반환하세요.
답변은 "True" 또는 "False"만 출력하세요.
''')
# 프롬포트 조정 가능합니다.

chain = prompt | llm

answer = chain.invoke({'message': input_string})

answer.content
type(answer.content)

str

나의 질문은 금융과 상관이 있다.


In [10]:
path = Path(__file__).parents[0]/ 'testfile' /'hankyung_data (1).json'
with open(path, 'r', encoding = 'utf-8') as f:
    data = json.load(f)

NameError: name '__file__' is not defined

In [2]:
path1 = r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\testfile\hankyung_data (1).json'
with open(path1, 'r', encoding = 'utf-8') as f:
    data = json.load(f)

In [4]:
from langchain_core.documents import Document

In [8]:
docs = [Document(page_content = news_info['news_content'],metadata = {'title': news_info['news_title'],'time': news_info['news_first_upload_time']})
    for news_info in data]

In [9]:
docs[0]

Document(metadata={'title': '송재혁 삼성전자 CTO, 차기 반도체협회장 내정', 'time': '2025-02-04 18:29'}, page_content='<div class="article-body" id="articletxt" itemprop="articleBody">\n <figure class="article-figure img-rt">\n  <div class="figure-img">\n   <img alt="송재혁 삼성전자 CTO, 차기 반도체협회장 내정" src="https://img.hankyung.com/photo/202502/AA.39405703.1.jpg"/>\n  </div>\n </figure>\n 송재혁 삼성전자 디바이스솔루션(DS)부문 최고기술책임자(CTO·사장·사진)가 차기 한국반도체산업협회장으로 내정됐다. 4일 산업계에 따르면 최근 삼성전자는 한국반도체산업협회에 송 사장을 차기 협회장으로 추천했다. 협회는 다음달 초 열리는 정기총회에서 송 사장의 회장 선출을 확정할 계획이다.\n <br/>\n <br/>\n 반도체산업협회장은 1991년 김광호 초대 회장 이후 삼성전자와 SK하이닉스의 사장급 경영진이 번갈아 맡아오고 있다. 이번엔 송 사장이 단일 후보로 올라 차기 협회장으로 선출될 가능성이 크다. 현재 반도체협회 회장을 맡고 있는 곽노정 SK하이닉스 사장(CEO) 임기는 이달 말까지다.\n <br/>\n <br/>\n 황정수 기자 hjs@hankyung.com\n</div>\n')

In [12]:
docs[0].page_content

'<div class="article-body" id="articletxt" itemprop="articleBody">\n <figure class="article-figure img-rt">\n  <div class="figure-img">\n   <img alt="송재혁 삼성전자 CTO, 차기 반도체협회장 내정" src="https://img.hankyung.com/photo/202502/AA.39405703.1.jpg"/>\n  </div>\n </figure>\n 송재혁 삼성전자 디바이스솔루션(DS)부문 최고기술책임자(CTO·사장·사진)가 차기 한국반도체산업협회장으로 내정됐다. 4일 산업계에 따르면 최근 삼성전자는 한국반도체산업협회에 송 사장을 차기 협회장으로 추천했다. 협회는 다음달 초 열리는 정기총회에서 송 사장의 회장 선출을 확정할 계획이다.\n <br/>\n <br/>\n 반도체산업협회장은 1991년 김광호 초대 회장 이후 삼성전자와 SK하이닉스의 사장급 경영진이 번갈아 맡아오고 있다. 이번엔 송 사장이 단일 후보로 올라 차기 협회장으로 선출될 가능성이 크다. 현재 반도체협회 회장을 맡고 있는 곽노정 SK하이닉스 사장(CEO) 임기는 이달 말까지다.\n <br/>\n <br/>\n 황정수 기자 hjs@hankyung.com\n</div>\n'

In [2]:
def preprocess_text(text):
    # 텍스트 전처리 / string으로 받아서 string으로 반환합니다.

    # URL 제거
    text = re.sub(r"http\S+", "", text)

    # 특수 문자 제거 (기본 구두점은 보존)
    text = re.sub(r"[^\w\s.,!?]", "", text)

    # 소문자 변환
    text = text.lower()

    # 공백 정리
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [3]:
input_string = '반도체 시장에 대한 뉴스를 알려줘'

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY_sesac')

llm = ChatOpenAI(
    model_name='gpt-4o',
    api_key=api_key
)

# mongoDB에서 데이터를 가져오기
# cluster = MongoClient(os.environ.get("mongo")) # 클러스터 
# db = cluster[''] # 유저 정보
# collection = db[""] # 내가 지정한 컬렉션 이름

# 지금은 임의로 데이터를 넣어놨지만 실제로는 데이터를 가져와야한다.
path = r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\testfile\hankyung_data (1).json'
with open(path, 'r', encoding = 'utf-8') as f:
    data = json.load(f)

# 다른 여러 파일들을 가져오려고 했으나 전처리가 되어있지 않아 사용 불가 따로 적용해서 찾아보겠음.


# 데이터를 Document로 변환
docs = [Document(page_content = news_info['news_content'],metadata = {'title': news_info['news_title'],'time': news_info['news_first_upload_time']})
    for news_info in data]

# 위와 같은 이유로 전처리하는 것을 찾아보겠음.

# 찾아보니까 임베딩 모델이 openaiembeddings를 사용하고 있어서 토큰화가 필요없다. 자동으로 하긴하는데, 전처리를 해주는 것은 좋아보인다.
# 전처리를 하고 임베딩 모델에 넣어서 faiss에 저장해보자.

# 전처리 과정 >> 함수로 정의해 두었습니다다
for i in range(len(docs)):
    docs[i].page_content = preprocess_text(docs[i].page_content)

# 청크로 나누기 
splitter = CharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
split_texts = splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# faiss에 저장
vector_store = FAISS.from_documents(split_texts, embeddings)


C:\Users\RMARKET\AppData\Local\Temp\ipykernel_10704\2307771759.py:41: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [4]:
# retriever 생성
# 유사도 검색을 코사인 유사도가 아닌 다른 방법으로 하고 싶다면 as.retriever의 인자를 바꾸면 된다.
retriever = vector_store.as_retriever()

system_prompt = ''' 
    Use the given context to answer the question. If you don't know the answer, say you don't know.  
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
      '''
    
#페르소나 조정 가능합니다.

prompt = ChatPromptTemplate.from_messages(
[
    ("system", system_prompt),
    ("human", input_string),
]
)

# llm과 prompt 연결결
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# 리트리버 연결결
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": input_string})

answer = chain.invoke({"input": input_string})

print(answer.content)

AttributeError: 'dict' object has no attribute 'content'

In [6]:
answer['answer']

'라이칭더 대만 총통 당선인은 반도체 산업 투자를 확대하겠다는 방침을 밝히며, 재료 및 장비 연구개발, IC 설계, 제조, 패키징, 테스트 분야에서 적극 지원하겠다고 강조했습니다. 대만의 투자 확대와 미국과의 협력 강화가 한국 반도체 산업에 타격이 될 수 있다는 우려가 있으며, TSMC가 주도하는 파운드리 분야에서 대만이 앞서 있는 것으로 평가되고 있습니다. 중국이 공급망을 봉쇄할 경우 대만 반도체 산업에 영향을 미칠 것이라는 전망도 존재하지만, 대만경제연구원은 중국이 이를 압박하지 않을 것이라고 분석했습니다.'

In [4]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY_sesac')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_9020\1037700103.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [7]:
# RAG 제작

input_string = '반도체 시장에 대한 뉴스를 알려줘'

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY_sesac')

llm = ChatOpenAI(
    model_name='gpt-4o',
    api_key=api_key
)

# mongoDB에서 데이터를 가져오기
cluster = MongoClient(os.environ.get("mongo")) # 클러스터 
db = cluster['Document'] # 유저 정보
newsdata = db["newsdata"] # 내가 지정한 컬렉션 이름

newsdatas = newsdata.find().to_list()

docs = [Document(page_content = news_info['news_content'],metadata = {
                                                                        'title': news_info['news_title'],
                                                                        'first_time': news_info['news_first_upload_time'],
                                                                        'last_time':news_info['news_last_upload_time'],
                                                                        'subcategory':news_info['news_category'],
                                                                        'major category':news_info['note'],
                                                                        'website':news_info['news_website'],
                                                                        'url':news_info['news_url'],
                                                                        'author':news_info['news_author'],
                                                                        })
    for news_info in newsdatas]


# 위와 같은 이유로 전처리하는 것을 찾아보겠음.

# 찾아보니까 임베딩 모델이 openaiembeddings를 사용하고 있어서 토큰화가 필요없다. 자동으로 하긴하는데, 전처리를 해주는 것은 좋아보인다.
# 전처리를 하고 임베딩 모델에 넣어서 faiss에 저장해보자.

# 청크로 나누기 
splitter = CharacterTextSplitter(chunk_size = 50, chunk_overlap = 5)
split_texts = splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# faiss에 저장

if not os.path.isfile(r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB\NewsData.faiss'):
    vector_store = FAISS.from_documents(split_texts, embeddings)

    vector_store.save_local(r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB',index_name = 'NewsData')

vector_store = FAISS.load_local(r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB',embeddings,allow_dangerous_deserialization=True,index_name = 'NewsData')






# retriever 생성
# 유사도 검색을 코사인 유사도가 아닌 다른 방법으로 하고 싶다면 as.retriever의 인자를 바꾸면 된다.
retriever = vector_store.as_retriever()

system_prompt = ''' 
Use the given context to answer the question. If you don't know the answer, say you don't know.  
"If you don't know the answer, say you don't know. "
"Use three sentence maximum and keep the answer concise. "
"Context: {context}"
    '''
#페르소나 조정 가능합니다.

prompt = ChatPromptTemplate.from_messages(
[
    ("system", system_prompt),
    ("human", "{input}"),
]
)

# llm과 prompt 연결결
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# 리트리버 연결결
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": input_string})

answer = chain.invoke({"input": input_string})
answer['answer']

Created a chunk of size 539, which is longer than the specified 50
Created a chunk of size 205, which is longer than the specified 50
Created a chunk of size 233, which is longer than the specified 50
Created a chunk of size 267, which is longer than the specified 50
Created a chunk of size 402, which is longer than the specified 50
Created a chunk of size 146, which is longer than the specified 50
Created a chunk of size 1530, which is longer than the specified 50
Created a chunk of size 2234, which is longer than the specified 50
Created a chunk of size 208, which is longer than the specified 50
Created a chunk of size 1565, which is longer than the specified 50
Created a chunk of size 1166, which is longer than the specified 50
Created a chunk of size 250, which is longer than the specified 50
Created a chunk of size 173, which is longer than the specified 50
Created a chunk of size 171, which is longer than the specified 50
Created a chunk of size 203, which is longer than the spec

'최근 반도체 시장은 트럼프의 발언이 주가에 큰 영향을 미치고 있는 것으로 보입니다. 특히 대만과의 반도체 동맹에 대한 우려로 인해 관련 주식들이 하락세를 보였으며, 미국과 중국의 경기 침체 가능성이 이러한 하락세를 더욱 가중시키고 있습니다. 그러나 전문가들 사이에서는 현재의 조정이 과도하다는 의견과 함께 반도체 주에 대한 투자 의견이 엇갈리고 있습니다.'

In [5]:
vector_store = FAISS.load_local(r'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB',embeddings,allow_dangerous_deserialization=True)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB\index.faiss for reading: No such file or directory